In [ ]:
using Pkg
Pkg.add("Plots")
Pkg.add("Colors")

In [ ]:
using Plots
using Colors

"Set the default plot size to something that fits a cell"
default(size = (800, 300))

In [ ]:
"Enumerate possible states of a single plant"
@enum InfectionStatus uninfected infected dead recovered immune

"Data structure containing the infection status of a plant"
mutable struct Plant
    status::InfectionStatus
    infection_time::Int8
end

"Parameters for a simulation"
mutable struct Parameters
    infection_rate::Float32
    reinfection_rate::Float32
    death_probability::Float32
    recovery_time::Int8
    immunity_rate::Float32
end

"Fast spread with low lethality and high immunity rate"
parameters = Parameters(0.2, 0.001, 0.02, 8, 0.1)

In [ ]:
"Create a 2D array of plants with 4 infected plants in the middle"
function create_map(width::Int64=32, height::Int64=32, immunity_rate::Float32=0.0)
    plants = Array{Plant}(undef, width, height)
    for i in 1:size(plants)[1]
        for j in 1:size(plants)[2]
            if rand(1)[1] < immunity_rate
                plants[i,j] = Plant(immune, 0)
            else
                plants[i,j] = Plant(uninfected, 0)
            end
        end
    end
    plants[width÷2,height÷2].status = infected
    plants[width÷2+1,height÷2].status = infected
    plants[width÷2,height÷2+1].status = infected
    plants[width÷2+1,height÷2+1].status = infected
    return plants
end

"Map the plants to colors for plotting"
function to_colors(plant::Plant)
    if plant.status == uninfected
        return RGB(0.0,0.8,0.0)
    end
    if plant.status == immune
        return RGB(0.2,0.2,1.0)
    end
    if plant.status == infected
        return RGB(0.8,0.0,0.0)
    end
    if plant.status == dead
        return RGB(0.1,0.1,0.1)
    end
    if plant.status == recovered
        return RGB(0.0,0.0,0.8)
    end
end

"""
Run the interaction between one plant and a neighbour.

If the neighbour is infected, it infect this plant with the propability
parameters.infection_rate or, if this plant is recovered, parameters.reinfection_rate.
"""
function interact!(new_plant::Plant, other_plant::Plant, parameters::Parameters)
    if new_plant.status == uninfected && other_plant.status == infected
        if rand(1)[1] < parameters.infection_rate
            new_plant.status = infected
            new_plant.infection_time = 0
        end
    end
    if new_plant.status == recovered && other_plant.status == infected
        if rand(1)[1] < parameters.reinfection_rate
            new_plant.status = infected
            new_plant.infection_time = 0
        end
    end
end

"""
Update a single plant, not accounting for it's interactions with the neighbours.
"""
function update!(new_plant::Plant, parameters::Parameters)
    if new_plant.status == infected
        new_plant.infection_time += 1
        if new_plant.infection_time > parameters.recovery_time
            new_plant.status = recovered
        end
        if rand(1)[1] < parameters.death_probability
            new_plant.status = dead
        end
    end
end

"""
Update the plants in the 2D array of Plants, using given parameters.
"""
function update(plants::Matrix{Plant}, parameters::Parameters)
    new_plants = deepcopy(plants)
    for i in 1:size(plants)[1]
        for j in 1:size(plants)[2]
            update!(new_plants[i,j], parameters)
        end
    end
    for i in 1:size(plants)[1]-1
        for j in 1:size(plants)[2]
            interact!(new_plants[i,j], plants[i+1,j], parameters)
            interact!(new_plants[i+1,j], plants[i,j], parameters)
        end
    end
    for i in 1:size(plants)[1]
        for j in 1:size(plants)[2]-1
            interact!(new_plants[i,j], plants[i,j+1], parameters)
            interact!(new_plants[i,j+1], plants[i,j], parameters)
        end
    end
    return new_plants
end

"Count the current number of infections"
function count_infections(plants::Matrix{Plant})
    infections = 0
    for i in 1:size(plants)[1]
        for j in 1:size(plants)[2]
            if plants[i,j].status == infected
                infections += 1
            end
        end
    end
    return infections
end

"Count the number of dead plants"
function count_deaths(plants::Matrix{Plant})
    deaths = 0
    for i in 1:size(plants)[1]
        for j in 1:size(plants)[2]
            if plants[i,j].status == dead
                deaths += 1
            end
        end
    end
    return deaths
end

In [ ]:
"The map of plants (a 2D array)"
plants = create_map(64, 64, parameters.infection_rate)
"An array of the infection counts at each time step"
infections = [count_infections(plants)]
"An array of the death counts at each time step"
deaths = [count_deaths(plants)]

"Function that runs the animation loop. (It's kind of heavy, so we want to compile it.)"
function animation(plants)
    "Build the animation frames by running simulation steps and generating a plot"
    anim = @animate for i ∈ 1:200
        plants = update(plants, parameters)
        append!(infections, count_infections(plants))
        append!(deaths, count_deaths(plants))
    
        l = @layout [a b]
        p1 = plot(to_colors.(plants),legend=false, border=:none)
        p2 = plot([infections, deaths], label = ["Infections" "Deaths"])
        frame = plot(p1, p2, layout = l)
        frame
    end
    return anim
end

gif(animation(plants), "pandemic.gif", fps = 5)